<a href="https://colab.research.google.com/github/dung18520632/Truy-van-da-phuong-tien/blob/main/POS%20Tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [915]:
import sys
import math
from decimal import *
import codecs
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
tag_list = set()
tag_count = {}
word_set = set()
fin="/content/Test.txt"

In [916]:
#Xử lí dữ liệu. Loại bỏ các dấu xuống dòng
def parse_traindata():
    wordtag_list = []
    try:
        input_file = codecs.open(fin, mode = 'r', encoding="utf-8")
        lines = input_file.readlines()
        for line in lines:
            line = line.strip('\r\n')
            data = line.split(" ")
            wordtag_list.append(data)
        input_file.close()
        print("parse_traindata:",wordtag_list)
        return wordtag_list

    except IOError:
        print("Không thể đọc file")
        sys.exit()

In [917]:
# Thống kê tần số xuất hiện của các tag đi với nhau! 
#Example:UN~tag~NN
def transition_count():
    global tag_list
    global word_set
    train_data = parse_traindata()
    transition_dict = {}
    global tag_count
    for value in train_data:
        previous = "start"
        for data in value:
            i = data[::-1]
            word = data[:-i.find("/") - 1]
            word_set.add(word.lower())
            data = data.split("/")
            tag = data[-1]
            tag_list.add(tag)
            if tag in tag_count:
                tag_count[tag] += 1
            else:
                tag_count[tag] = 1
            if (previous + "~tag~" + tag) in transition_dict:
                transition_dict[previous + "~tag~" + tag] += 1
                previous = tag
            else:
                transition_dict[previous + "~tag~" + tag] = 1
                previous = tag
    return transition_dict


In [918]:
#Tính xác suất chuyển trạng thái smoothing có trong Data, sử dụng Laplace +1
def transition_probability():
    count_dict = transition_count()
    print("transition_count:",count_dict)
    prob_dict = {}
    
    for key in count_dict:
        den = 0
        val = key.split("~tag~")[0]
        for key_2 in count_dict:
            if key_2.split("~tag~")[0] == val:
                den += count_dict[key_2]
        prob_dict[key] = Decimal(count_dict[key]+1)/(den+len(tag_list))
    return prob_dict,count_dict

In [919]:
#Tính xác suất chuyển trạng thái smoothing KHÔNG có trong Data, sử dụng Laplace +1
def transition_smoothing():
    a={}
    transition_prob,count_dict = transition_probability()
    
    for i in count_dict.items():
        arr=i[::-1]
        if arr[1].split("~tag~")[0] in a:
            a[arr[1].split("~tag~")[0]]+=arr[0]
        else:
            a[arr[1].split("~tag~")[0]]=arr[0]
    for j in tag_list:
        if j not in a.keys():
            a[j]=0
    for tag in tag_list:
        if "start" +  "~tag~" +tag not in  transition_prob:
            transition_prob[("start" + "~tag~" + tag)] = Decimal(1) / Decimal(a["start"]+len(tag_list))
    for tag1 in tag_list:
        for tag2 in tag_list:
            if (tag1 +"~tag~" + tag2) not in transition_prob:
                transition_prob[(tag1+"~tag~"+tag2)] = Decimal(1)/Decimal(a[tag1]+len(tag_list))
    print("Xác suất chuyển trạng thái:",transition_prob)
    return transition_prob

In [920]:
#Tính tần số xuất hiện của các word/tag
#Example:Con/UN
def emission_count():
    #print "In Emission Model"
    train_data = parse_traindata()
    count_word = {}
    for value in train_data:
        for data in value:
            i = data[::-1]
            word = data[:-i.find("/") - 1]
            tag = data.split("/")[-1]
            if word.lower() + "/" + tag in count_word:
                count_word[word.lower() + "/" + tag] +=1
            else:
                count_word[word.lower() + "/" + tag] = 1
    return count_word

In [921]:
#Xác suất thể hiện trạng thái ẩn(emission), sử dụng Laplace +1
def emission_probability():
    global tag_count
    word_count = emission_count()
    print("emission_count:",word_count)
    emission_prob_dict = {}
    for key in word_count:
        emission_prob_dict[key] = Decimal(word_count[key]+1)/(tag_count[key.split("/")[-1]]+len(word_set))
    for word in word_set:
        for tag in tag_count.items():
            if word+"/"+tag[0] not in emission_prob_dict:
                emission_prob_dict[word+"/"+tag[0]]=Decimal(1)/(tag[1]+len(word_set))
    print("emission_probability:",emission_prob_dict)
    return emission_prob_dict


In [922]:
#Thuật toán viterbi
def viterbi_algorithm(sentence, tags, transition_prob, emission_prob,tag_count_emis, word_set):
    word_list = sentence.split(" ")
    current_prob = {}
    for tag in tags:
        tp = Decimal(0)
        em = Decimal(0)
        if "start~tag~"+tag in transition_prob:
            tp = Decimal(transition_prob["start~tag~"+tag])
        if word_list[0].lower() in word_set:
            if (word_list[0].lower()+"/"+tag) in emission_prob:
                em = Decimal(emission_prob[word_list[0].lower()+"/"+tag])
                current_prob[tag] = tp * em
        else:
            em = Decimal(1) /(tag_count_emis[tag] +len(word_set))
            current_prob[tag] = tp
    if len(word_list) == 1:
        max_path = max(current_prob, key=current_prob.get)
        return max_path
    else:
        for i in range(1, len(word_list)):
            previous_prob = current_prob
            current_prob = {}
            locals()['dict{}'.format(i)] = {}
            previous_tag = ""
            for tag in tags:
                if word_list[i].lower() in word_set:
                    if word_list[i].lower()+"/"+tag in emission_prob:
                        em = Decimal(emission_prob[word_list[i].lower()+"/"+tag])
                        max_prob, previous_state = max((Decimal(previous_prob[previous_tag]) * Decimal(transition_prob[previous_tag + "~tag~" + tag]) * em, previous_tag) for previous_tag in previous_prob)
                        current_prob[tag] = max_prob
                        print(tag)
                        print("previous_state",previous_state)
                        print(current_prob)
                        locals()['dict{}'.format(i)][previous_state + "~" + tag] = max_prob
                        previous_tag = previous_state
                else:
                    em = Decimal(1) /(tag_count_emis[tag] +len(word_set))
                    max_prob, previous_state = max((Decimal(previous_prob[previous_tag]) * Decimal(transition_prob[previous_tag+"~tag~"+tag]) *em, previous_tag) for previous_tag in previous_prob)
                    current_prob[tag] = max_prob
                    locals()['dict{}'.format(i)][previous_state + "~" + tag] = max_prob
                    previous_tag = previous_state
            if i == len(word_list)-1:
                max_path = ""
                last_tag = max(current_prob, key=current_prob.get)
                max_path = max_path + last_tag 
                for j in range(len(word_list)-1,0,-1):
                    for key in locals()['dict{}'.format(j)]:
                        data = key.split("~")
                        if data[-1] == previous_tag:
                            max_path = max_path + " " +data[0]
                            previous_tag = data[0]
                            break
                result = max_path.split()
                result.reverse()
                return " ".join(result)

In [923]:
#Xác suất chuyển trạng thái
transition_model = transition_smoothing()

parse_traindata: [['con/UN', 'mèo/NN', 'trèo/VB', 'cây/UN', 'cau/NN'], ['con/UN', 'chuột/NN', 'trèo/VB', 'cây/UN', 'cau/NN'], ['con/UN', 'chuột/NN', 'hỏi/VB', 'con/UN', 'mèo/NN'], ['con/UN', 'trèo/VB', 'là/VB', 'con/UN', 'nào/PRP']]
transition_count: {'start~tag~UN': 4, 'UN~tag~NN': 6, 'NN~tag~VB': 3, 'VB~tag~UN': 4, 'UN~tag~VB': 1, 'VB~tag~VB': 1, 'UN~tag~PRP': 1}
Xác suất chuyển trạng thái: {'start~tag~UN': Decimal('0.625'), 'UN~tag~NN': Decimal('0.5833333333333333333333333333'), 'NN~tag~VB': Decimal('0.5714285714285714285714285714'), 'VB~tag~UN': Decimal('0.5555555555555555555555555556'), 'UN~tag~VB': Decimal('0.1666666666666666666666666667'), 'VB~tag~VB': Decimal('0.2222222222222222222222222222'), 'UN~tag~PRP': Decimal('0.1666666666666666666666666667'), 'start~tag~NN': Decimal('0.125'), 'start~tag~VB': Decimal('0.125'), 'start~tag~PRP': Decimal('0.125'), 'UN~tag~UN': Decimal('0.08333333333333333333333333333'), 'NN~tag~UN': Decimal('0.1428571428571428571428571429'), 'NN~tag~NN': Dec

In [924]:
#Xác suất emission
emission_model = emission_probability()

parse_traindata: [['con/UN', 'mèo/NN', 'trèo/VB', 'cây/UN', 'cau/NN'], ['con/UN', 'chuột/NN', 'trèo/VB', 'cây/UN', 'cau/NN'], ['con/UN', 'chuột/NN', 'hỏi/VB', 'con/UN', 'mèo/NN'], ['con/UN', 'trèo/VB', 'là/VB', 'con/UN', 'nào/PRP']]
emission_count: {'con/UN': 6, 'mèo/NN': 2, 'trèo/VB': 3, 'cây/UN': 2, 'cau/NN': 2, 'chuột/NN': 2, 'hỏi/VB': 1, 'là/VB': 1, 'nào/PRP': 1}
emission_probability: {'con/UN': Decimal('0.4117647058823529411764705882'), 'mèo/NN': Decimal('0.2'), 'trèo/VB': Decimal('0.2857142857142857142857142857'), 'cây/UN': Decimal('0.1764705882352941176470588235'), 'cau/NN': Decimal('0.2'), 'chuột/NN': Decimal('0.2'), 'hỏi/VB': Decimal('0.1428571428571428571428571429'), 'là/VB': Decimal('0.1428571428571428571428571429'), 'nào/PRP': Decimal('0.2'), 'là/UN': Decimal('0.05882352941176470588235294118'), 'là/NN': Decimal('0.06666666666666666666666666667'), 'là/PRP': Decimal('0.1'), 'cau/UN': Decimal('0.05882352941176470588235294118'), 'cau/VB': Decimal('0.0714285714285714285714285714

In [925]:
    tag_count_emis={}
    for prob in emission_model.items():
         i=prob[::-1]
         i=list(i)
         key_tag=i[1]
         val=key_tag.split("/")[-1]
         if val in tag_count_emis:
             tag_count_emis[val]+=1
         else:
             tag_count_emis[val]=1
    fin_train = "/content/gg.txt"
    input_file = codecs.open(fin_train, mode='r', encoding="utf-8")
    #Kết quả được ghi vào file HmmOutput.txt để predict
    fout = codecs.open("HmmOutput.txt",mode='w',encoding="utf-8")
    print("--------------------------------------------------------------------")
    print("Thuật toán ViterBi")
    for sentence in input_file.readlines():
        path = viterbi_algorithm(sentence, tag_list, transition_model, emission_model,tag_count_emis, word_set)
        sentence=sentence.strip("\r\n")
        word=sentence.split(" ")
        tag = path.split(" ")
        for j in range(0,len(word)):
            if j == len(word)-1:
                
                fout.write(word[j] + "/" + tag[j]+u'\n')
            else:
                fout.write(word[j] + "/" + tag[j] + " ")
    

--------------------------------------------------------------------
Thuật toán ViterBi
UN
previous_state UN
{'UN': Decimal('0.001261534025374855824682814302')}
NN
previous_state UN
{'UN': Decimal('0.001261534025374855824682814302'), 'NN': Decimal('0.01000816993464052287581699346')}
VB
previous_state UN
{'UN': Decimal('0.001261534025374855824682814302'), 'NN': Decimal('0.01000816993464052287581699346'), 'VB': Decimal('0.01225490196078431372549019608')}
PRP
previous_state UN
{'UN': Decimal('0.001261534025374855824682814302'), 'NN': Decimal('0.01000816993464052287581699346'), 'VB': Decimal('0.01225490196078431372549019608'), 'PRP': Decimal('0.004289215686274509803921568628')}
UN
previous_state VB
{'UN': Decimal('0.0004004869921824939125977188262')}
NN
previous_state NN
{'UN': Decimal('0.0004004869921824939125977188262'), 'NN': Decimal('0.00009531590413943355119825708060')}
VB
previous_state NN
{'UN': Decimal('0.0004004869921824939125977188262'), 'NN': Decimal('0.0000953159041394335511982

In [926]:
Result = codecs.open("HmmOutput.txt",mode='r',encoding="utf-8")
for i in Result.readlines():
    print(i)

con/UN trèo/NN là/VB con/UN mèo/NN

